In [1]:
import os
import sys
import pandas as pd
import childespy
import numpy as np
from os.path import join, exists

Reinstalling childesr version 0.2.1
--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Brussels) [https]
 8: Brazil (PR) [https]
 9: Brazil (RJ) [https]
10: Brazil (SP 1) [https]
11: Brazil (SP 2) [https]
12: Bulgaria [https]
13: Canada (MB) [https]
14: Canada (ON 3) [https]
15: Chile (Santiago) [https]
16: China (Beijing 2) [https]
17: China (Beijing 3) [https]
18: China (Hefei) [https]
19: China (Hong Kong) [https]
20: China (Guangzhou) [https]
21: China (Lanzhou) [https]
22: China (Nanjing) [https]
23: China (Shanghai 2) [https]
24: China (Shenzhen) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: Ecuador (Quito) [https]
33: France (Lyon 1

R[write to console]: trying URL 'https://cran.case.edu/bin/macosx/contrib/4.2/childesr_0.2.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 75307 bytes (73 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con


The downloaded binary packages are in
	/var/folders/g7/6fmhhq2x5813j_xlfxrqxln80000gn/T//RtmpjvjT2T/downloaded_packages


In [2]:
sys.path.append('../../')
from src.utils import split_gen, data_cleaning, load_splits, configuration, paths
config = configuration.Config()
np.random.seed(config.SEED)

/Users/nicolewong/opt/anaconda3/envs/child_grammar/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generate Train, Val for All and Age splits

In [3]:
#get all of the North American and British English adult and child utterances without xxx or yyy
#concatenate them at the the transcript level
#hold out 20% for validation 

In [4]:
corpora = childespy.get_sql_query('select * from corpus where \
collection_name in ("Eng-NA", "Eng-UK") and data_source = "CHILDES"')
corpora

R[write to console]: Using current database version: '2021.1'.



,id,name,collection_name,data_source,collection_id
1,34,Garvey,Eng-NA,CHILDES,2
2,35,Valian,Eng-NA,CHILDES,2
3,36,Bernstein,Eng-NA,CHILDES,2
4,37,Clark,Eng-NA,CHILDES,2
5,38,PetersonMcCabe,Eng-NA,CHILDES,2
...,...,...,...,...,...
59,229,Wells,Eng-UK,CHILDES,12
60,230,Gathburn,Eng-UK,CHILDES,12
61,231,Nuffield,Eng-UK,CHILDES,12
62,232,Lara,Eng-UK,CHILDES,12


In [5]:
childes_datasets = ",".join([str(x) for x in corpora.collection_id])

In [7]:
save_path = join(config.finetune_dir, 'utt_glosses.pkl')

if config.regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, target_child_name, corpus_name, target_child_age, type from utterance where collection_name in ("Eng-NA", "Eng-UK") \
    and collection_id in ('+childes_datasets+') and speaker_code in ("MOT", "FAT","CHI")' , db_version = "2020.1")
    utt_glosses.to_pickle(save_path)
else: 
    utt_glosses = pd.read_pickle(save_path)

R[write to console]: Using supported database version: '2020.1'.



In [8]:
utt_glosses = utt_glosses[utt_glosses.gloss != ''] # Added 8/16/21 -- drop empty utterances

all_utt_glosses = utt_glosses.rename(columns = {'id' : 'utterance_id'}).copy()
# end both cites

## "all" split

In [9]:
import imp
imp.reload(paths)
imp.reload(data_cleaning)
imp.reload(split_gen)

/var/folders/g7/6fmhhq2x5813j_xlfxrqxln80000gn/T/ipykernel_14416/3753350811.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'src.utils.split_gen' from '/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/split_gen.py'>

In [10]:
# expectations for verbose output.
# 232 should be (4205071, 7) (because you added two extra fields)
# 233 likewise is (3959952, 7)

# Save the partition markings as well, for verification purposes.

In [11]:
success_sample_path = paths.get_sample_csv_path('training', 'Providence', 'all', 'success', age = None, n=5000)

In [12]:
success_sample_path

'/Users/nicolewong/projects/child-grammar/output/experiments/full_scale/sample/n=5000/Providence_all/training_success_utts_5000.csv'

### No tags

In [13]:
data = data_cleaning.drop_errors(all_utt_glosses)
data_no_tags = data_cleaning.clean_glosses_and_add_speaker_label(data, use_tags = False)
assert not any(['xxx' in s for s in set(data_no_tags.gloss_with_punct)])
assert not any(['yyy' in s for s in set(data_no_tags.gloss_with_punct)])
data_no_tags['tokens'] = [str(x).lower().split(' ') for x in data_no_tags.gloss]

/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.gloss = [fix_gloss(x) for x in data.gloss]
/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['punct'] = [punct_for_type[x] if x in punct_for_type else fill_punct_val
/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:194: SettingWithCopyWarning: 

In [14]:
Providence_notags_split_folder = paths.get_directory({
    "task_phase" : 'extract_data',
    "training_split" : 'Providence',
    "training_dataset" : 'all',    
    "test_split" : None,
    "test_dataset" : None,
    "model_type" : None,
    "context_width" : None,
    "n_samples" : config.n_across_time,
    "task_name" : None,
    "use_tags" : False
})
Providence_notags_split_folder

'/Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_no_tags'

In [15]:
notags_split_glosses_df, notags_train_df = split_gen.exec_split_gen(
    data_no_tags, Providence_notags_split_folder, 'train')

/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_no_tags/train.txt


/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_no_tags/eval.txt
Writing split glosses to: /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_no_tags/data_pool_with_phases.pkl


### With tags

In [16]:
data = data_cleaning.drop_errors(all_utt_glosses)
data_with_tags = data_cleaning.clean_glosses_and_add_speaker_label(data, use_tags = True)
assert not any(['xxx' in s for s in set(data_with_tags.gloss_with_punct)])
assert not any(['yyy' in s for s in set(data_with_tags.gloss_with_punct)])
data_with_tags['tokens'] = [str(x).lower().split(' ') for x in data_with_tags.gloss]

/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.gloss = [fix_gloss(x) for x in data.gloss]
/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['punct'] = [punct_for_type[x] if x in punct_for_type else fill_punct_val
/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:194: SettingWithCopyWarning: 

In [17]:
Providence_withtags_split_folder = paths.get_directory({
    "task_phase" : 'extract_data',
    "training_split" : 'Providence',
    "training_dataset" : 'all',    
    "test_split" : None,
    "test_dataset" : None,
    "model_type" : None,
    "context_width" : None,
    "n_samples" : config.n_across_time,
    "task_name" : None,
    "use_tags" : True
})
Providence_withtags_split_folder

'/Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_with_tags'

In [18]:
withtags_split_glosses_df, withtags_train_df = split_gen.exec_split_gen(
    data_with_tags, Providence_withtags_split_folder, 'train')

/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_with_tags/train.txt


/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_with_tags/eval.txt
Writing split glosses to: /Users/nicolewong/projects/child-grammar/output/experiments/full_scale/extract_data/n=500/Providence_all_with_tags/data_pool_with_phases.pkl


In [19]:
from datetime import datetime
print(datetime.today())

2022-11-19 16:56:03.439555
